In [1]:
import os
import logging
import pandas as pd
import boto3
import tempfile
import xgboost as xgb
from io import BytesIO

from dotenv import load_dotenv
from pathlib import Path

In [4]:
load_dotenv()

aws_access_key_id = os.getenv('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.getenv('AWS_SECRET_ACCESS_KEY')
aws_region = os.getenv('AWS_REGION')

s3 = boto3.client(
    service_name='s3',
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=aws_region
)

In [8]:
bucket = 'mlflow-bucket-diabetes'

response = s3.list_objects_v2(Bucket=bucket)

In [9]:
response

{'ResponseMetadata': {'RequestId': 'CR7HK3K1JPHXD79R',
  'HostId': 'yzyspXW1tyo+ONVokR/7AxnWKTC6tJor3d06d8GziHFsBjdxC4/tRTAS/68KHp4jMO5doN00z/1yX62sM8tlLJ+WFEcsaQgT',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'yzyspXW1tyo+ONVokR/7AxnWKTC6tJor3d06d8GziHFsBjdxC4/tRTAS/68KHp4jMO5doN00z/1yX62sM8tlLJ+WFEcsaQgT',
   'x-amz-request-id': 'CR7HK3K1JPHXD79R',
   'date': 'Thu, 17 Jul 2025 15:20:20 GMT',
   'x-amz-bucket-region': 'eu-north-1',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'IsTruncated': False,
 'Contents': [{'Key': '2/',
   'LastModified': datetime.datetime(2025, 7, 17, 15, 8, 8, tzinfo=tzutc()),
   'ETag': '"d41d8cd98f00b204e9800998ecf8427e"',
   'ChecksumAlgorithm': ['CRC64NVME'],
   'ChecksumType': 'FULL_OBJECT',
   'Size': 0,
   'StorageClass': 'STANDARD'},
  {'Key': '2/6/',
   'LastModified': datetime.datetime(2025, 7, 17, 15, 8, 15, tzinfo=tzutc()),
   'ETag': '"d41d8cd98f00b204e98009

In [12]:
response['Contents']

[{'Key': '2/',
  'LastModified': datetime.datetime(2025, 7, 17, 15, 8, 8, tzinfo=tzutc()),
  'ETag': '"d41d8cd98f00b204e9800998ecf8427e"',
  'ChecksumAlgorithm': ['CRC64NVME'],
  'ChecksumType': 'FULL_OBJECT',
  'Size': 0,
  'StorageClass': 'STANDARD'},
 {'Key': '2/6/',
  'LastModified': datetime.datetime(2025, 7, 17, 15, 8, 15, tzinfo=tzutc()),
  'ETag': '"d41d8cd98f00b204e9800998ecf8427e"',
  'ChecksumAlgorithm': ['CRC64NVME'],
  'ChecksumType': 'FULL_OBJECT',
  'Size': 0,
  'StorageClass': 'STANDARD'},
 {'Key': '2/6/y_train.parquet',
  'LastModified': datetime.datetime(2025, 7, 17, 15, 8, 48, tzinfo=tzutc()),
  'ETag': '"fed0c901622417866724a53de008090f"',
  'ChecksumAlgorithm': ['CRC64NVME'],
  'ChecksumType': 'FULL_OBJECT',
  'Size': 1360,
  'StorageClass': 'STANDARD'}]

In [14]:
last_modified = sorted(response['Contents'], key=lambda obj: obj['LastModified'], reverse=True)

In [17]:
last_modified[0]['Key']

'2/6/y_train.parquet'